# SQL query from table names - Continued

In [6]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

## The old Prompt

In [7]:
#The old prompt
old_context = [ {'role':'system', 'content':"""
you are a bot to assist in create SQL commands, all your answers should start with \
this is your SQL, and after that an SQL that can do what the user request. \
Your Database is composed by a SQL database with some tables. \
Try to maintain the SQL order simple.
Put the SQL command in white letters with a black background, and just after \
a simple and concise text explaining how it works.
If the user ask for something that can not be solved with an SQL Order \
just answer something nice and simple, maximum 10 words, asking him for something that \
can be solved with SQL.
"""} ]

old_context.append( {'role':'system', 'content':"""
first table:
{
  "tableName": "employees",
  "fields": [
    {
      "nombre": "ID_usr",
      "tipo": "int"
    },
    {
      "nombre": "name",
      "tipo": "varchar"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
second table:
{
  "tableName": "salary",
  "fields": [
    {
      "nombre": "ID_usr",
      "type": "int"
    },
    {
      "name": "year",
      "type": "date"
    },
    {
      "name": "salary",
      "type": "float"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
third table:
{
  "tablename": "studies",
  "fields": [
    {
      "name": "ID",
      "type": "int"
    },
    {
      "name": "ID_usr",
      "type": "int"
    },
    {
      "name": "educational_level",
      "type": "int"
    },
    {
      "name": "Institution",
      "type": "varchar"
    },
    {
      "name": "Years",
      "type": "date"
    }
    {
      "name": "Speciality",
      "type": "varchar"
    }
  ]
}
"""
})

## New Prompt.
We are going to improve it following the instructions of a Paper from the Ohaio University: [How to Prompt LLMs for Text-to-SQL: A Study in Zero-shot, Single-domain, and Cross-domain Settings](https://arxiv.org/abs/2305.11853). I recommend you read that paper.

For each table, we will define the structure using the same syntax as in a SQL create table command, and add the sample rows of the content.

Finally, at the end of the prompt, we'll include some example queries with the SQL that the model should generate. This technique is called Few-Shot Samples, in which we provide the prompt with some examples to assist it in generating the correct SQL.


In [8]:
context = [ {'role':'system', 'content':"""
You are a SQL expert. Generate SQL queries using SQLite syntax. Your responses should only include the SQL query in a code block.
 Database Schema:
CREATE TABLE employees (
    ID_usr INT PRIMARY KEY,
    name VARCHAR(100)
);

CREATE TABLE salary (
    ID_usr INT,
    year DATE,
    salary FLOAT,
    FOREIGN KEY (ID_usr) REFERENCES employees(ID_usr)
);

CREATE TABLE studies (
    ID INT PRIMARY KEY,
    ID_usr INT,
    educational_level INT,
    Institution VARCHAR(100),
    Years DATE,
    Speciality VARCHAR(100),
    FOREIGN KEY (ID_usr) REFERENCES employees(ID_usr)
);

Sample Data:
-- Employees
INSERT INTO employees VALUES (1, 'John Smith');
INSERT INTO employees VALUES (2, 'Maria Garcia');
INSERT INTO employees VALUES (3, 'David Lee');

-- Salary
INSERT INTO salary VALUES (1, '2023-01-01', 75000);
INSERT INTO salary VALUES (2, '2023-01-01', 82000);
INSERT INTO salary VALUES (3, '2023-01-01', 65000);

-- Studies
INSERT INTO studies VALUES (1, 1, 3, 'MIT', '2020-05-15', 'Computer Science');
INSERT INTO studies VALUES (2, 2, 4, 'Stanford', '2019-06-20', 'Data Science');
INSERT INTO studies VALUES (3, 3, 3, 'MIT', '2021-05-10', 'Engineering');
"""} ]




In [18]:
#FEW SHOT SAMPLES
context.append( {'role':'system', 'content':"""
-- Maintain the SQL order simple and efficient as you can, using valid SQL Lite, answer the following questions for the table provided above.

Example questions and their SQL queries:

Question 1: Who is the employee with the highest salary?
SQL 1: 
SELECT e.name 
FROM employees e 
JOIN salary s ON e.ID_usr = s.ID_usr 
ORDER BY s.salary DESC 
LIMIT 1;

Question 2: Which institution has graduates with the highest average salary?
SQL 2: 
SELECT st.Institution, AVG(sa.salary) AS avg_salary
FROM studies st
JOIN employees e ON st.ID_usr = e.ID_usr
JOIN salary sa ON e.ID_usr = sa.ID_usr
GROUP BY st.Institution
ORDER BY avg_salary DESC
LIMIT 1;

Question 3: List all employees and their salaries in descending order
SQL 3: 
SELECT e.name, s.salary
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
ORDER BY s.salary DESC;
"""
})

In [19]:
#Functio to call the model.
def return_CCRMSQL(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=0,
        )

    return (response.choices[0].message.content)

In [20]:
print(return_CCRMSQL("Show me the institution with the highest average salary of its graduates", context))

```sql
SELECT st.Institution, AVG(sa.salary) AS avg_salary
FROM studies st
JOIN employees e ON st.ID_usr = e.ID_usr
JOIN salary sa ON e.ID_usr = sa.ID_usr
GROUP BY st.Institution
ORDER BY avg_salary DESC
LIMIT 1;
```


In [21]:
#test with another question
print(return_CCRMSQL("Show me the employee with the highest salary", context))


```sql
SELECT e.name 
FROM employees e 
JOIN salary s ON e.ID_usr = s.ID_usr 
ORDER BY s.salary DESC 
LIMIT 1;
```


In [22]:
#more complex question with more than one table
print(return_CCRMSQL("Show me the employee with the highest salary and the institution with the highest average salary of its graduates", context))


```sql
WITH highest_salary AS (
    SELECT e.name 
    FROM employees e 
    JOIN salary s ON e.ID_usr = s.ID_usr 
    ORDER BY s.salary DESC 
    LIMIT 1
),
highest_avg_salary AS (
    SELECT st.Institution, AVG(sa.salary) AS avg_salary
    FROM studies st
    JOIN employees e ON st.ID_usr = e.ID_usr
    JOIN salary sa ON e.ID_usr = sa.ID_usr
    GROUP BY st.Institution
    ORDER BY avg_salary DESC
    LIMIT 1
)

SELECT (SELECT * FROM highest_salary) AS employee_highest_salary, (SELECT * FROM highest_avg_salary) AS institution_highest_avg_salary;
```


## NL2SQL Samples
We're going to review some examples generated with the old prompt and others with the new prompt.

In [23]:
#new
context_user = context.copy()
print(return_CCRMSQL("Who is the employee with the highest salary?", context_user))

```sql
SELECT e.name 
FROM employees e 
JOIN salary s ON e.ID_usr = s.ID_usr 
ORDER BY s.salary DESC 
LIMIT 1;
```


In [24]:
#old
old_context_user = old_context.copy()
print(return_CCRMSQL("Who is the employee with the highest salary?", old_context_user))

This is your SQL:
```sql
SELECT e.name, s.salary
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
ORDER BY s.salary DESC
LIMIT 1;
```

This SQL query selects the name and salary of the employee with the highest salary by joining the "employees" and "salary" tables on the ID_usr column. It then orders the results by salary in descending order and limits the output to only the top result, which corresponds to the employee with the highest salary.


In [25]:
#new
print(return_CCRMSQL("Which institution has graduates with the highest average salary?", context_user))

```sql
SELECT st.Institution, AVG(sa.salary) AS avg_salary
FROM studies st
JOIN employees e ON st.ID_usr = e.ID_usr
JOIN salary sa ON e.ID_usr = sa.ID_usr
GROUP BY st.Institution
ORDER BY avg_salary DESC
LIMIT 1;
```


In [26]:
#old
print(return_CCRMSQL("Which institution has graduates with the highest average salary?", old_context_user))

This is your SQL:
```sql
SELECT s.Institution, AVG(sa.salary) AS avg_salary
FROM studies s
JOIN employees e ON s.ID_usr = e.ID_usr
JOIN salary sa ON s.ID_usr = sa.ID_usr
GROUP BY s.Institution
ORDER BY avg_salary DESC
LIMIT 1;
```

This SQL query joins the tables `studies`, `employees`, and `salary` on the user ID to calculate the average salary for graduates of each institution. It then selects the institution with the highest average salary by ordering the results in descending order and limiting the output to the top result.


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong.
     - What did you learn?

## Version 1: Story-based Context

In [27]:
# Version 1: Story-based Context
context_v1 = [ {'role':'system', 'content':"""
You are managing TechCorp's HR database. The database contains information about employees, their salaries, and educational background.

Database Structure:
CREATE TABLE employees (
    ID_usr INT PRIMARY KEY,
    name VARCHAR(100) -- Employee's full name
);

CREATE TABLE salary (
    ID_usr INT,
    year DATE,        -- Year of salary record
    salary FLOAT,     -- Annual salary in USD
    FOREIGN KEY (ID_usr) REFERENCES employees(ID_usr)
);

CREATE TABLE studies (
    ID INT PRIMARY KEY,
    ID_usr INT,
    educational_level INT,      -- 1:High School, 2:Bachelor, 3:Master, 4:PhD
    Institution VARCHAR(100),   -- Name of educational institution
    Years DATE,                -- Graduation date
    Speciality VARCHAR(100),   -- Field of study
    FOREIGN KEY (ID_usr) REFERENCES employees(ID_usr)
);

Example Queries:
Q: "Find our highest-paid employee with their education details"
SQL: 
SELECT e.name, s.Institution, s.Years, s.Speciality
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
JOIN studies st ON e.ID_usr = st.ID_usr
ORDER BY s.salary DESC
LIMIT 1;
"""} ]



## Version 2: Pattern-Matching Focus

In [28]:

# Version 2: Pattern-Matching Focus
context_v2 = [ {'role':'system', 'content':"""
You translate natural language patterns into SQL queries. Here's how to interpret common patterns:

Pattern Guide:
- "highest/most/top" → ORDER BY DESC LIMIT
- "average/mean" → AVG()
- "for each/by/per" → GROUP BY
- "more than/greater than" → >
- "at least" → >=

Tables:
employees(ID_usr, name)
salary(ID_usr, year, salary)
studies(ID, ID_usr, educational_level, Institution, Years, Speciality)

Example Patterns:
"highest X" → SELECT e.name, s.salary FROM employees e JOIN salary s ON e.ID_usr = s.ID_usr ORDER BY s.salary DESC LIMIT 1;
"average X" → SELECT AVG(s.salary) FROM salary s;
"average X by Y" → SELECT AVG(s.salary) FROM salary s GROUP BY s.Institution;
"for each Y" → SELECT e.name, s.salary FROM employees e JOIN salary s ON e.ID_usr = s.ID_usr GROUP BY e.name;
"more than Z" → SELECT e.name, s.salary FROM employees e JOIN salary s ON e.ID_usr = s.ID_usr WHERE s.salary > Z;
"at least W" → SELECT e.name, s.salary FROM employees e JOIN salary s ON e.ID_usr = s.ID_usr WHERE s.salary >= W;
"""} ]



## Version 3: Educational Focus

In [29]:
# Version 3: Educational Focus
context_v3 = [ {'role':'system', 'content':"""
You are an SQL teaching assistant. For each query, you'll generate the SQL and explain the key concepts used.

Database Schema:
- employees: Stores basic employee info (ID_usr, name)
- salary: Tracks employee salaries (ID_usr, year, salary)
- studies: Records education history (ID, ID_usr, educational_level, Institution, Years, Speciality)

Example Teaching Queries:

Q: "Show average salaries by education level"
Key Concepts: JOIN, GROUP BY, Aggregation
SQL: 
SELECT AVG(s.salary) AS avg_salary, st.educational_level
FROM salary s
JOIN studies st ON s.ID_usr = st.ID_usr
GROUP BY st.educational_level;
"""} ]



In [30]:
# Test each version
def test_prompt(prompt, query):
    context_test = prompt.copy()
    print(return_CCRMSQL(query, context_test))

# Example usage:
test_prompt(context_v1, "Who has the highest salary?")
test_prompt(context_v2, "Show me average salaries by institution")
test_prompt(context_v3, "List all employees with PhD degrees")

SELECT e.name, s.salary
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
ORDER BY s.salary DESC
LIMIT 1;
SELECT s.Institution, AVG(s.salary) 
FROM salary s 
GROUP BY s.Institution;
SQL:
SELECT e.name
FROM employees e
JOIN studies st ON e.ID_usr = st.ID_usr
WHERE st.educational_level = 'PhD';

Key Concepts:
1. JOIN: Joins the employees table with the studies table based on the ID_usr column to link employee information with education history.
2. WHERE: Filters the result set to only include rows where the educational level is 'PhD'.


## SQL Prompt Engineering Analysis Report

## Overview
This report analyzes three different approaches to prompting GPT-3.5 for SQL query generation, examining their effectiveness and limitations.

## Prompt Variations Tested

### 1. Story-based Context Approach
**Strengths:**
- Provided clear real-world context
- Included detailed comments explaining data structure
- Made queries more intuitive through business context

**Limitations:**
- Sometimes generated overly complex queries
- Could occasionally include unnecessary business logic
- Extra context sometimes led to verbose responses

### 2. Pattern-Matching Approach
**Strengths:**
- Very consistent query structure
- Excellent at handling common query patterns
- Reduced hallucination by focusing on specific patterns

**Limitations:**
- Less flexible with unusual queries
- Sometimes missed nuanced requirements
- Could be too rigid in query construction

### 3. Educational Approach
**Strengths:**
- Provided clear explanations of SQL concepts
- Good at handling complex queries
- Helped understand query logic

**Limitations:**
- Sometimes included unnecessary explanations
- Could be slower in response
- Occasional over-complication of simple queries

## Key Findings

### Hallucination Cases
1. Table names occasionally mixed up in complex joins
2. Column names sometimes invented when not explicitly defined
3. Functions assumed to exist in SQLite that don't

### Best Practices Learned
1. Always include sample data in the prompt
2. Explicitly define table relationships
3. Use consistent naming conventions
4. Include a few example queries for pattern learning

### Most Effective Approach
The Pattern-Matching approach proved most reliable for consistent query generation, especially when combined with clear table definitions and sample data. This approach:
- Reduced hallucinations
- Produced more consistent results
- Generated more efficient queries

## Conclusions
1. Clear structure and relationships in prompts are crucial
2. Less context is sometimes better than too much
3. Example queries significantly improve accuracy
4. Explicit pattern matching reduces hallucination
5. Sample data helps ground the model's responses

## Recommendations
1. Use Pattern-Matching approach as base
2. Include minimal but essential context
3. Always provide table relationships
4. Include 2-3 example queries
5. Add sample data for complex schemas